In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [3]:
df = pd.read_csv('Combined Data.csv')

In [4]:
oh = OneHotEncoder(sparse_output=False)
encoder = oh.fit_transform(df[['status']])

encoded_df = pd.DataFrame(encoder, columns=oh.get_feature_names_out(['status']))

In [5]:
df['statement'] = df['statement'].str.replace('[^a-zA-Z]+', ' ', regex=True)

In [6]:
df['statement'] = df['statement'].apply(lambda x: str(x).lower())

In [7]:
vectorizer = TextVectorization(max_tokens=50000, output_mode='int', output_sequence_length=1800)

In [8]:
vectorizer.adapt(df['statement'])

In [9]:
X = vectorizer(df['statement'])

In [10]:
X

<tf.Tensor: shape=(53043, 1800), dtype=int64, numpy=
array([[ 585,    6, 4760, ...,    0,    0,    0],
       [ 920,  630,  943, ...,    0,    0,    0],
       [  33,  268,   96, ...,    0,    0,    0],
       ...,
       [  12,   68,   86, ...,    0,    0,    0],
       [1244, 1433,  967, ...,    0,    0,    0],
       [   2,   13,   63, ...,    0,    0,    0]])>

In [11]:
y = np.array(encoded_df)

In [12]:
y = tf.convert_to_tensor(y)

In [13]:
dataset = tf.data.Dataset.from_tensor_slices((X, y))

In [14]:
dataset = dataset.batch(32)
dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
dataset = dataset.shuffle(len(dataset))
dataset = dataset.cache()

In [15]:
dataset_length = len(dataset)
train_size = int(dataset_length * 0.7)  # 70% for training
val_size = int(dataset_length * 0.2)  # 15% for validation
test_size = dataset_length - (train_size + val_size)  # 

train_ds = dataset.take(train_size)
val_ds = dataset.skip(train_size).take(val_size)
test_ds = dataset.skip(train_size+val_size).take(test_size)

In [16]:
model = Sequential()
model.add(Embedding(50000+1, 32))
model.add(Bidirectional(LSTM(32, activation='tanh')))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(7, activation='sigmoid'))

model.compile(loss="BinaryCrossentropy", optimizer='Adam')

model.fit(train_ds, epochs=10, validation_data=val_ds)

Epoch 1/10
1159/1160 ━━━━━━━━━━━━━━━━━━━━ 0s 381ms/step - loss: 0.3196

2024-12-23 13:35:11.406541: W tensorflow/core/kernels/data/cache_dataset_ops.cc:913] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


1160/1160 ━━━━━━━━━━━━━━━━━━━━ 457s 391ms/step - loss: 0.3195 - val_loss: 0.2207
Epoch 2/10


2024-12-23 13:35:23.990752: W tensorflow/core/kernels/data/cache_dataset_ops.cc:913] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


1159/1160 ━━━━━━━━━━━━━━━━━━━━ 0s 395ms/step - loss: 0.2170

2024-12-23 13:43:02.289070: W tensorflow/core/kernels/data/cache_dataset_ops.cc:913] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


1160/1160 ━━━━━━━━━━━━━━━━━━━━ 471s 406ms/step - loss: 0.2169 - val_loss: 0.1955
Epoch 3/10


2024-12-23 13:43:14.984175: W tensorflow/core/kernels/data/cache_dataset_ops.cc:913] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


 566/1160 ━━━━━━━━━━━━━━━━━━━━ 41:47 4s/step - loss: 0.2013

KeyboardInterrupt: 

In [38]:
text = vectorizer('finished for the hearts intention to watch the series i even thought about it im worried')

prediction = model.predict(np.expand_dims(text, 0))

np.round(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


array([[0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [36]:
df.statement[197]

'finished for the heart s intention to watch the series i even thought about it i m worried'

In [ ]:
y